# Bitcoin Graph Analysis

In [ ]:
import os
import csv
import sqlite3

import igraph
import pandas as pd

from time_manager import get_unixtime

In [ ]:
IGRAPH_PICKLE = ''
g = igraph.Graph.Read_Pickle(IGRAPH_PICKLE)

In [ ]:
partition = g.community_leiden(objective_function='modularity')

df = pd.DataFrame()
df['membership'] = partition.membership

dfcnt = df['membership'].value_counts()
largest_comm = dfcnt.sort_values(ascending=False).index